In [1]:
import torch
from transformers import TapasForQuestionAnswering
from transformers import TapasTokenizer
from transformers import pipeline
import pandas as pd

2023-08-18 15:21:50.997081: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-18 15:21:51.043936: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-18 15:21:51.253749: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-18 15:21:51.257577: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 15:21:52.390549: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
model = TapasForQuestionAnswering.from_pretrained('google/tapas-large-finetuned-wtq')
tokenizer = TapasTokenizer.from_pretrained("google/tapas-large-finetuned-wtq")

data = {"Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], "Number of movies": ["87", "53", "69"]}
table = pd.DataFrame.from_dict(data)
question = "how many movies does Leonardo Di Caprio have?"

tqa = pipeline(task="table-question-answering", model=model, tokenizer=tokenizer)
print(tqa(table=table, query=question)['cells'][0])

53


In [4]:
# Define the input shape
batch_size = 1
sequence_length = 29

# Modify the input shape of the dummy_input dictionary
dummy_input = {
    "input_ids": torch.zeros((batch_size, sequence_length), dtype=torch.long),
    "attention_mask": torch.zeros((batch_size, sequence_length), dtype=torch.long),
    "token_type_ids": torch.zeros((batch_size, sequence_length, 7), dtype=torch.long),
    #"attention_mask_for_classification": torch.zeros((batch_size, sequence_length), dtype=torch.float),
}

torch.onnx.export(
    model=model,
    args=(dummy_input,),
    f="tapas.onnx",
    input_names=list(dummy_input.keys()),
    output_names=["output"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "token_type_ids": {0: "batch_size", 1: "sequence_length", 2: "num_labels" },
        #"attention_mask_for_classification": {0: "batch_size", 1: "sequence_length"},
        #"output": {0: "batch_size", 1: "sequence_length", 2: "num_labels"},
    },
)

/home/amokrov/.virtualenvs/my_openvino_notebooks/lib/python3.8/site-packages/transformers/models/tapas/modeling_tapas.py:1601: TracerWarning: torch.as_tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  self.indices = torch.as_tensor(indices)
/home/amokrov/.virtualenvs/my_openvino_notebooks/lib/python3.8/site-packages/transformers/models/tapas/modeling_tapas.py:1602: TracerWarning: torch.as_tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  self.num_segments = torch.as_tensor(num_segments, device=indices.device)
/home/amokrov/.vi

UnsupportedOperatorError: Exporting the operator 'aten::scatter_reduce' to ONNX opset version 14 is not supported. Please feel free to request support or submit a pull request on PyTorch GitHub: https://github.com/pytorch/pytorch/issues

In [6]:
import openvino

openvino.convert_model(
    model,
    example_input=dummy_input
)

[ WARNING ]  Please fix your imports. Module %s has been moved to %s. The old module will be deleted in version %s.


OpConversionFailure: Check 'is_conversion_successful' failed at src/frontends/pytorch/src/frontend.cpp:138:
FrontEnd API failed with OpConversionFailure:
Model wasn't fully converted. Failed operations detailed log:
-- prim::Constant with a message:
String constant cannot be converted to OpenVINO opset and should be removed by consuming operation.
-- prim::ListConstruct with a message:
Exception happened during conversion of operation __module.model/prim::ListConstruct_1422 with schema (no schema)
Check 'c_node' failed at src/frontends/pytorch/src/op/list_construct.cpp:25:
FrontEnd API failed with OpConversionFailure:
Translation for prim::ListConstruct support only constant inputs

-- prim::ListUnpack with a message:
prim::ListUnpack: unsupported input node: util::PtFrameworkNode __module.model/aten::broadcast_tensors_1424 (__module.model/prim::ListConstruct_1423[0]:dynamic[...]) -> (dynamic[...])
Summary:
-- No conversion rule found for operations: aten::broadcast_tensors, aten::logi

In [5]:
# Define the input shape
batch_size = 1
sequence_length = 29

# Modify the input shape of the dummy_input dictionary
dummy_input = {
    "input_ids": torch.zeros((batch_size, sequence_length), dtype=torch.long),
    "token_type_ids": torch.zeros((batch_size, sequence_length, 7), dtype=torch.long),
    "position_ids": torch.zeros((batch_size, sequence_length), dtype=torch.long),
    #"inputs_embeds": torch.zeros((batch_size, sequence_length, 1024), dtype=torch.float),
}

torch.onnx.export(
    model=model,
    args=(dummy_input,),
    f="tapas.onnx",
    input_names=list(dummy_input.keys()),
    output_names=["output"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "token_type_ids": {0: "batch_size", 1: "sequence_length", 2: "num_labels"},
        "position_ids": {0: "batch_size", 1: "sequence_length"},
        #"inputs_embeds": {0: "batch_size", 1: "sequence_length", 2: "num_labels"},
        #"output": {0: "batch_size", 1: "sequence_length", 2: "num_labels"},
    },
)

UnsupportedOperatorError: Exporting the operator 'aten::scatter_reduce' to ONNX opset version 14 is not supported. Please feel free to request support or submit a pull request on PyTorch GitHub: https://github.com/pytorch/pytorch/issues

In [7]:
import openvino

openvino.convert_model(
    model,
    example_input=dummy_input
)

OpConversionFailure: Check 'is_conversion_successful' failed at src/frontends/pytorch/src/frontend.cpp:138:
FrontEnd API failed with OpConversionFailure:
Model wasn't fully converted. Failed operations detailed log:
-- prim::Constant with a message:
String constant cannot be converted to OpenVINO opset and should be removed by consuming operation.
-- prim::ListConstruct with a message:
Exception happened during conversion of operation __module.model/prim::ListConstruct_1422 with schema (no schema)
Check 'c_node' failed at src/frontends/pytorch/src/op/list_construct.cpp:25:
FrontEnd API failed with OpConversionFailure:
Translation for prim::ListConstruct support only constant inputs

-- prim::ListUnpack with a message:
prim::ListUnpack: unsupported input node: util::PtFrameworkNode __module.model/aten::broadcast_tensors_1424 (__module.model/prim::ListConstruct_1423[0]:dynamic[...]) -> (dynamic[...])
Summary:
-- No conversion rule found for operations: aten::broadcast_tensors, aten::logi